# Build a variational autoencoder

Based on the ability to generate UK car registration numbers, one can build a dataset of training and test data. With a suitable vectorizer, one can go ahead and make an autoencoder.

The decoder part of the variational encoder can then be used to generate new reg numbers.

### References

https://blog.keras.io/building-autoencoders-in-keras.html

https://towardsdatascience.com/understanding-variational-autoencoders-vaes-f70510919f73

In [1]:
import sys
import numpy as np
import keras
from keras import regularizers
from keras import layers
from keras import backend
from keras import optimizers
import random

if '..' not in sys.path:
    sys.path.append('..')
from car_reg_generator.car_reg_generator.uk_reg import UkRegGenerator
from car_reg_generator.car_reg_generator.uk_reg import UkRegDvlaVectorizer

### Data generation

In [2]:
n_train = 10000
n_val = 1000
n_test = 1000

random.seed(0)
g = UkRegGenerator()
v = UkRegDvlaVectorizer()

train_strs = [g.get_reg() for _ in range(n_train)]
train_vecs = np.array([v.vectorize(x) for x in train_strs])
val_strs = [g.get_reg() for _ in range(n_val)]
val_strs = [x for x in val_strs if x not in set(train_strs)]  # sanity check
val_vecs = np.array([v.vectorize(x) for x in val_strs])
test_strs = [g.get_reg() for _ in range(n_test)]
test_strs = [x for x in test_strs if x not in set(train_strs) | set(val_strs)]  # sanity check
test_vecs = np.array([v.vectorize(x) for x in test_strs])

vec_length = len(train_vecs[0])
print('input vector length = ' + str(vec_length))
print('actual number of validation samples = ' + str(len(val_strs)))
print('actual number of test samples = ' + str(len(test_strs)))

input vector length = 150
actual number of validation samples = 1000
actual number of test samples = 1000


### Variational autoencoder

Design the architecture

In [3]:
latent_dim = 15

### Define the encoder
inputs = keras.Input(shape=(vec_length,))
h = inputs
h = layers.Dense(vec_length, activation='relu')(h)
h = layers.Dense(50, activation='relu')(h)
h = layers.Dense(25, activation='relu')(h)
z_mean = layers.Dense(latent_dim)(h)
z_log_sigma = layers.Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = backend.random_normal(shape=(backend.shape(z_mean)[0], latent_dim),
                                    mean=0., stddev=0.1)
    return z_mean + backend.exp(z_log_sigma) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_sigma])

encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')

### Define the decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = latent_inputs
x = layers.Dense(25, activation='relu')(x)
x = layers.Dense(50, activation='relu')(x)
x = layers.Dense(vec_length, activation='relu')(x)
x = layers.Dense(vec_length, activation='sigmoid')(x)
outputs = x
decoder = keras.Model(latent_inputs, outputs, name='decoder')

outputs = decoder(encoder(inputs)[2])
vae = keras.Model(inputs, outputs, name='vae_mlp')

### Define the loss function
reconstruction_loss = keras.losses.binary_crossentropy(inputs, outputs)
reconstruction_loss *= vec_length
kl_loss = 1 + z_log_sigma - backend.square(z_mean) - backend.exp(z_log_sigma)
kl_loss = backend.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = backend.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)

Do some training

In [4]:
opt = optimizers.Adamax(learning_rate=0.004)
vae.compile(optimizer=opt, loss='binary_crossentropy')
vae.fit(train_vecs, train_vecs,
        epochs=200,
        batch_size=8,
        shuffle=True,
        validation_data=(val_vecs, val_vecs))

opt = optimizers.Adamax(learning_rate=0.001)
vae.compile(optimizer=opt, loss='binary_crossentropy')
vae.fit(train_vecs, train_vecs,
        epochs=200,
        batch_size=8,
        shuffle=True,
        validation_data=(val_vecs, val_vecs))

Epoch 1/200
1250/1250 [==============================] - 1s 1ms/step - loss: 28.2511 - val_loss: 26.6473
Epoch 2/200
1250/1250 [==============================] - 1s 932us/step - loss: 25.5185 - val_loss: 24.3848
Epoch 3/200
1250/1250 [==============================] - 1s 893us/step - loss: 23.2336 - val_loss: 22.4293
Epoch 4/200
1250/1250 [==============================] - 1s 899us/step - loss: 21.9204 - val_loss: 21.7983
Epoch 5/200
1250/1250 [==============================] - 1s 913us/step - loss: 21.2705 - val_loss: 21.3612
Epoch 6/200
1250/1250 [==============================] - 1s 900us/step - loss: 20.8199 - val_loss: 21.0122
Epoch 7/200
1250/1250 [==============================] - 1s 898us/step - loss: 20.4580 - val_loss: 20.5835
Epoch 8/200
1250/1250 [==============================] - 1s 916us/step - loss: 20.0721 - val_loss: 20.2651
Epoch 9/200
1250/1250 [==============================] - 1s 920us/step - loss: 19.6486 - val_loss: 19.7655
Epoch 10/200
1250/1250 [===============

1250/1250 [==============================] - 1s 888us/step - loss: 9.6320 - val_loss: 10.6184
Epoch 78/200
1250/1250 [==============================] - 1s 873us/step - loss: 9.6522 - val_loss: 10.1244
Epoch 79/200
1250/1250 [==============================] - 1s 884us/step - loss: 9.6017 - val_loss: 10.3266
Epoch 80/200
1250/1250 [==============================] - 1s 876us/step - loss: 9.4617 - val_loss: 10.1652
Epoch 81/200
1250/1250 [==============================] - 1s 883us/step - loss: 9.4050 - val_loss: 10.2698
Epoch 82/200
1250/1250 [==============================] - 1s 879us/step - loss: 9.3269 - val_loss: 10.0110
Epoch 83/200
1250/1250 [==============================] - 1s 873us/step - loss: 9.2717 - val_loss: 10.2363
Epoch 84/200
1250/1250 [==============================] - 1s 880us/step - loss: 9.1811 - val_loss: 10.6479
Epoch 85/200
1250/1250 [==============================] - 1s 880us/step - loss: 9.0823 - val_loss: 10.0517
Epoch 86/200
1250/1250 [==========================

Epoch 154/200
1250/1250 [==============================] - 1s 879us/step - loss: 5.9790 - val_loss: 6.6265
Epoch 155/200
1250/1250 [==============================] - 1s 875us/step - loss: 6.0431 - val_loss: 6.8019
Epoch 156/200
1250/1250 [==============================] - 1s 878us/step - loss: 5.9492 - val_loss: 6.5878
Epoch 157/200
1250/1250 [==============================] - 1s 877us/step - loss: 5.9892 - val_loss: 6.2476
Epoch 158/200
1250/1250 [==============================] - 1s 880us/step - loss: 5.9317 - val_loss: 6.5385
Epoch 159/200
1250/1250 [==============================] - 1s 878us/step - loss: 5.9667 - val_loss: 6.6790
Epoch 160/200
1250/1250 [==============================] - 1s 874us/step - loss: 5.9050 - val_loss: 6.8119
Epoch 161/200
1250/1250 [==============================] - 1s 882us/step - loss: 5.9187 - val_loss: 6.5284
Epoch 162/200
1250/1250 [==============================] - 1s 888us/step - loss: 5.8872 - val_loss: 7.3363
Epoch 163/200
1250/1250 [============

Epoch 31/200
1250/1250 [==============================] - 1s 871us/step - loss: 4.1442 - val_loss: 4.8642
Epoch 32/200
1250/1250 [==============================] - 1s 874us/step - loss: 4.1567 - val_loss: 4.7782
Epoch 33/200
1250/1250 [==============================] - 1s 870us/step - loss: 4.1684 - val_loss: 4.6957
Epoch 34/200
1250/1250 [==============================] - 1s 870us/step - loss: 4.1590 - val_loss: 4.7626
Epoch 35/200
1250/1250 [==============================] - 1s 871us/step - loss: 4.1495 - val_loss: 4.7172
Epoch 36/200
1250/1250 [==============================] - 1s 876us/step - loss: 4.1427 - val_loss: 4.9358
Epoch 37/200
1250/1250 [==============================] - 1s 878us/step - loss: 4.1281 - val_loss: 4.8064
Epoch 38/200
1250/1250 [==============================] - 1s 872us/step - loss: 4.1060 - val_loss: 4.7208
Epoch 39/200
1250/1250 [==============================] - 1s 876us/step - loss: 4.1371 - val_loss: 4.9101
Epoch 40/200
1250/1250 [======================

1250/1250 [==============================] - 1s 876us/step - loss: 3.9492 - val_loss: 4.5706
Epoch 109/200
1250/1250 [==============================] - 1s 872us/step - loss: 3.9601 - val_loss: 4.5075
Epoch 110/200
1250/1250 [==============================] - 1s 874us/step - loss: 3.9452 - val_loss: 4.5993
Epoch 111/200
1250/1250 [==============================] - 1s 867us/step - loss: 3.9826 - val_loss: 4.4959
Epoch 112/200
1250/1250 [==============================] - 1s 875us/step - loss: 3.9436 - val_loss: 4.5265
Epoch 113/200
1250/1250 [==============================] - 1s 887us/step - loss: 3.9347 - val_loss: 4.5558
Epoch 114/200
1250/1250 [==============================] - 1s 872us/step - loss: 3.9333 - val_loss: 4.5339
Epoch 115/200
1250/1250 [==============================] - 1s 884us/step - loss: 3.9489 - val_loss: 4.5876
Epoch 116/200
1250/1250 [==============================] - 1s 878us/step - loss: 3.9453 - val_loss: 4.5282
Epoch 117/200
1250/1250 [==========================

1250/1250 [==============================] - 1s 871us/step - loss: 3.8296 - val_loss: 4.4884
Epoch 185/200
1250/1250 [==============================] - 1s 868us/step - loss: 3.8454 - val_loss: 4.4410
Epoch 186/200
1250/1250 [==============================] - 1s 869us/step - loss: 3.8675 - val_loss: 4.2829
Epoch 187/200
1250/1250 [==============================] - 1s 872us/step - loss: 3.8127 - val_loss: 4.3512
Epoch 188/200
1250/1250 [==============================] - 1s 869us/step - loss: 3.8615 - val_loss: 4.4712
Epoch 189/200
1250/1250 [==============================] - 1s 870us/step - loss: 3.8386 - val_loss: 4.4788
Epoch 190/200
1250/1250 [==============================] - 1s 870us/step - loss: 3.8241 - val_loss: 4.4483
Epoch 191/200
1250/1250 [==============================] - 1s 883us/step - loss: 3.8294 - val_loss: 4.4946
Epoch 192/200
1250/1250 [==============================] - 1s 873us/step - loss: 3.8343 - val_loss: 4.3509
Epoch 193/200
1250/1250 [==========================

Evaluation

In [5]:
decoded_regs = vae.predict(test_vecs)
print(test_strs[:10])
recovered = [v.recover(x) for x in decoded_regs]
print(recovered[:10])
acc = np.sum([x == y for x, y in zip(recovered, test_strs)]) / len(test_strs)
print('overall accuracy = ' + str(acc))
print('by digit accuracy:')
for i_digit in range(len(test_strs[0])):
    acc = np.sum([x[i_digit] == y[i_digit] for x, y in zip(recovered, test_strs)]) / len(test_strs)
    print('    ' + str(acc))

['MY33WSZ', 'WE83GXF', 'LU19JLV', 'KX41GVE', 'WP91NCV', 'SG47LFF', 'HB17WIY', 'CA89AXN', 'GT57AKA', 'CD77SJT']
['MY33WSZ', 'WE83GXI', 'LU19JLV', 'KX41GVE', 'WP91NCV', 'SG47LFF', 'HB47WIY', 'CA89AXN', 'GT57AKA', 'CD77SJT']
overall accuracy = 0.737
by digit accuracy:
    0.995
    0.984
    0.994
    0.991
    0.981
    0.99
    0.792


### Use decoder as generator

By directly sampling in the latent space, and running through the decoder, one effectively has a data generator.

In [6]:
[v.recover(x) for x in decoder.predict(np.random.random((5, latent_dim)))]

['HY01ZCT', 'CA01IAY', 'PG11NKA', 'MY01IXW', 'SG21BCJ']

One can do some sampling and look at some stats to see what kind of data comes out. For instance, the number of reg numbers with pair of integers below 50 and above 50

In [7]:
random_vectors = 10 * (np.random.random((1000, latent_dim)) - 0.5)
generated_regs = [v.recover(x) for x in decoder.predict(random_vectors)]
n_reg_below_50 = len({x for x in generated_regs if int(x[2:4]) < 50})
n_reg_above_50 = len({x for x in generated_regs if int(x[2:4]) >= 50})
print('reg numbers below 50 = ' + str(n_reg_below_50))
print('reg numbers above 50 = ' + str(n_reg_above_50))

reg numbers below 50 = 433
reg numbers above 50 = 567


### Avoid generating any training data

There is a chance that a random generation of data might produce one of the actual registration numbers used in training. However, any input registrations are represented as a probability distribution in the latent space; so one could require that any sample vector used in the latent space to be 'sufficiently far' down the tails of all the recorded distributions.

The distributions of the training data in the latent space are readily available:

In [8]:
# Get the distributions for each reg number in the data
all_strs = train_strs + val_strs + test_strs
all_vecs = np.array([v.vectorize(x) for x in all_strs])
means, log_sigmas, _ = encoder.predict(all_vecs)
distn_dict = {x: (y,z) for x, y, z in zip(all_strs, means, np.exp(log_sigmas))}  # dict<reg no: (mean, sigma)>
distn_dict['YK66BIQ']
# for a multivariate distribution ~ N(mean, Var) valued in the latent space, 
# mean is the first vector below, and Var is a diagonal matrix, the square root
# of the diagonal entries being given in the second vector.
# If the KL loss function has done its job, the distributions should be close to N(0,1)

(array([ 0.9664675 , -0.38055828, -0.48702085,  0.00808121, -0.02023169,
         0.26767403, -0.9605626 , -0.8348456 , -0.2087945 , -0.00560325,
         0.19209874, -0.06166378, -0.36049056,  0.3712803 , -0.39906198],
       dtype=float32),
 array([0.5190886 , 0.5172951 , 0.52408266, 1.0009084 , 0.9980573 ,
        0.5679686 , 0.5070283 , 0.49884477, 0.46346048, 0.49687105,
        0.5390688 , 0.9790228 , 0.5516896 , 0.5181226 , 0.5831158 ],
       dtype=float32))

One can access the distributions that contribute to the latent space, that represent numbers below 50:

In [9]:
distns_below_50 = {k: v for k, v in distn_dict.items() if int(k[2:4]) < 50}
print('number of distributions with number below 50 = ' + str(len(distns_below_50)))

number of distributions with number below 50 = 6103


and use these to find only random vectors that are 'sufficiently far away' from the means of those reg numbers with numbers below 50. 

In [10]:
random_vectors = 10 * (np.random.random((1000, latent_dim)) - 0.5)
min_mahalanobis_dists = [np.min([np.linalg.norm(np.abs(x-mu)/sigma) 
                                 for mu, sigma in distns_below_50.values()])
                         for x in random_vectors]

In [25]:
x = random_vectors[0]
means_mtr = np.array([mu for mu, sigma in distns_below_50.values()])
sigma_mtr = np.array([sigma for mu, sigma in distns_below_50.values()])
min_mahalanobis np.min(np.linalg.norm((x - means_mtr) / sigma_mtr, axis=1))
bob

15.090161261305113

In [13]:
mahalanobis_threshold = 1.0
filtered_random_vectors = [x for x, y in zip(random_vectors, np.array(min_mahalanobis_dists)/latent_dim) 
                           if y > mahalanobis_threshold]
print('number of filtered random vectors = ' + str(len(filtered_random_vectors)))

number of filtered random vectors = 669


In [12]:
filtered_generated_regs = [v.recover(x) for x in decoder.predict(np.array(filtered_random_vectors))]
filtered_n_reg_above_50 = len({x for x in filtered_generated_regs if int(x[2:4]) > 50})
print('filtered reg numbers above 50 = ' + str(filtered_n_reg_above_50))

filtered reg numbers above 50 = 401
